## Import neccessary packages

In [1]:
import rioxarray as rio
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
import matplotlib.pyplot as plt


## Use this section only to downscale from hourly to daily.

In [2]:
ds = xr.open_mfdataset('/media/sarat/Study/Evap_IMDAA/*.nc') # change the folder accordingly
# #####  temporal downscaling considering daily mean from all hourly values ##
ds_daily = ds.resample(time='1D').mean('time')

/home/sarat/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/sarat/anaconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


MergeError: conflicting values for variable 'EVARCA_sfc' on objects to be combined. You can skip this check by specifying compat='override'.

<ipython-input-18-90dac1e4ace2>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  vals  = cdo.fldsum(input=ifile,returnCdf=True).variables['rain'][:]


## Use this section to calculate total ET and conversion

In [3]:
# Loading daily data obtained earlier #
ds = xr.open_dataset('/media/sarat/Study/Evap_IMDAA/evap_daily.nc') # change the folder accordingly
## Tot ET = Soil Evap + Canopy EVap
ds_tot_et = ds.EVARSS_sfc+ds.EVARSS_sfc # Total Et in kg/m2/s
#####
### convert to mm/day ##
ds_tot_et = ds_tot_et*86400 # Tot ET in mm/day

## Use this section to extract the area masked nc file using shapefile 

In [5]:
## Load the shapefile corresponding to each sub-basin ###
### By default I loaded the whole Kaveri basin ###
fname = '/media/sarat/Study/Walter/Geospatial_DB/Geospatial_DB/cauvery_subbasins.shp' # change the file accordingly
shape = gpd.read_file(fname, crs='epsg:4326')

In [8]:
## mask the tot ET array using shapefile ###
ds_tot_et.rio.set_spatial_dims(x_dim='longitude',y_dim='latitude', inplace=True)
ds_tot_et.rio.write_crs("epsg:4326", inplace=True)
ds_et_mask=ds_tot_et.rio.clip(shape.geometry.apply(mapping), shape.crs, drop=False)

ERROR 1: PROJ: proj_create_from_database: /home/sarat/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


CRSError: PROJ: proj_create_from_database: /home/sarat/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.

In [7]:
#%% Converting the array to time-series ###
## using latitude weights for better areal averaging ###
weights = np.cos(np.deg2rad(ds_et_mask.latitude))
weights.name = 'weights'
ds_1d=ds_et_mask.weighted(weights)
ds_daily_wm = ds_1d.mean(('latitude','longitude'))
########
ds_daily_wm=ds_daily_wm.to_pandas()
####
ds_1d_wm = pd.Series(data=ds_daily_wm, name = 'ET (mm/day)')

NameError: name 'ds_et_mask' is not defined

In [9]:

#%% Plotting using default matplotlib ###
ds_1d_wm.plot(color='blue', ylabel='mm/day', xlabel='Time', 
              title='ET in Selected basin')


NameError: name 'ds_1d_wm' is not defined